In [4]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
df = pd.read_csv('googleplaystore.csv.csv')

# current time to check 
now = datetime.now()
current_hour = now.hour

# should the show graph between 4 PM and 6 PM  IST only
if current_hour >= 16 and current_hour < 18:
    
    # Apply filters
    filtered_df = df[
        (df['Rating'] >= 4.2) & 
        (~df['App'].str.contains(r'\d', na=False)) &  # No numbers in app name
        (df['Category'].str[0].isin(['T', 'P'])) &  # Categories starting with T or P
        (df['Reviews'] > 1000) &
        (df['Size'] >= 20000000) &  # 20 MB in bytes
        (df['Size'] <= 80000000)    # 80 MB in bytes
    ]
    
    # Converting Last Updated to datetime
    filtered_df['Date'] = pd.to_datetime(filtered_df['Last Updated'])
    filtered_df['Year_Month'] = filtered_df['Date'].dt.to_period('M')
    
    # Grouping by Year_Month and Category for cumulative installs
    category_time = filtered_df.groupby(['Year_Month', 'Category'])['Installs'].sum().reset_index()
    category_time['Year_Month'] = category_time['Year_Month'].astype(str)
    
    # Calculating cumulative installs for each category
    categories = category_time['Category'].unique()
    
    # Calculating month-over-month growth to highlight 25%+ increases
    category_time_sorted = category_time.sort_values(['Category', 'Year_Month'])
    category_time_sorted['Previous_Installs'] = category_time_sorted.groupby('Category')['Installs'].shift(1)
    category_time_sorted['Growth_Rate'] = (
        (category_time_sorted['Installs'] - category_time_sorted['Previous_Installs']) / 
        category_time_sorted['Previous_Installs'] * 100
    )
    
    # Translating category names in legend
    category_translations = {
        'Travel & Local': 'Voyage et Local',  # French
        'Productivity': 'Productividad',       # Spanish
        'Photography': '写真'                   # Japanese
    }
    
    # Creating stacked area chart
    fig = go.Figure()
    
    for category in categories:
        cat_data = category_time_sorted[category_time_sorted['Category'] == category]
        
        # Translate category name if it's one of the special ones
        display_name = category_translations.get(category, category)
        
        # Check if any month has 25%+ growth for color intensity
        has_high_growth = (cat_data['Growth_Rate'] > 25).any()
        
        # Set opacity based on growth (higher for categories with 25%+ growth)
        opacity = 0.9 if has_high_growth else 0.6
        
        fig.add_trace(go.Scatter(
            x=cat_data['Year_Month'],
            y=cat_data['Installs'],
            name=display_name,
            mode='lines',
            stackgroup='one',
            fillcolor=None,  # Will use default colors
            opacity=opacity,
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Month: %{x}<br>' +
                         'Installs: %{y:,}<br>' +
                         '<extra></extra>'
        ))
    
    # Update layout
    fig.update_layout(
        title='Cumulative App Installs Over Time by Category',
        xaxis=dict(
            title='Month',
            tickangle=-45
        ),
        yaxis=dict(
            title='Cumulative Installs',
            tickformat=','
        ),
        height=600,
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.02
        ),
        hovermode='x unified'
    )
    

    fig.show()
    
    
    high_growth = category_time_sorted[category_time_sorted['Growth_Rate'] > 25]
    if not high_growth.empty:
        print("\nMonths with 25%+ growth {highlighted with htime('%I:%M %p')}higher color intensity):")
        print(high_growth[['Year_Month', 'Category', 'Growth_Rate']].to_string(index=False))
    
else:
    print("This graph is only available between 4 PM and 6 PM IST")
    print(f"Current time: {now.strftime('%I:%M %p')}")

This graph is only available between 4 PM and 6 PM IST
Current time: 09:00 AM
